# Tasks

Provides API interfaces for managing the creation and modification of automation tasks. Tasks are used in workflows for automation. We will here go through the process of creating simple tasks. Workflows should be addressed later in an advanced GreenLake VMAAS API workshop.

/api/tasks

The above command returns JSON structured like this:

{
  "tasks": [
     {
      "id": 5,
      "accountId": 1,
      "name": "aptitude upgrade",
      "taskType": {
        "id": 1,
        "code": "script",
        "name": "Shell Script"
      },
      "file": {
        "id": 1275,
        "sourceType": "local",
        "contentRef": null,
        "contentPath": null,
        "repository": null,
        "content": "apt-get upgrade -y"
      },
      "taskOptions": {
      }
    },
  ],
  "meta": {
    "offset": 0,
    "max": 25,
    "size": 1,
    "total": 1
  }
}
This endpoint retrieves all tasks.

HTTP REQUEST
curl GET  "$API_URL/api/tasks

## Task Creation

In this first part, we will create two simple tasks:
- 1st task to run uptime command on the VM

- 2nd task to write a file on the file system. at the end of this lab, We will then make use of the snapshot feature to move back to the original state of the Virtual machine.


HTTP REQUEST
curl POST "$API_URL/api/tasks

JSON PARAMETERS

|Parameter |   Default  | Description  |
|---|---|---|
|  name |   | A unique name for the task  |
|  code |   |   | A unique code for the task
| taskType.code  |   | The type of task  |
|  taskOptions |   | Map of options specific to each type. eg. script  |
|  resultType |   |   |
|  executeTarget |   | The execution target. eg. local,remote,resource. The default value varies by task type.  |
|  retryable | false  | If the task should be retried or not.  |
|  retryCount |   | The number of times to retry.  |
|  retryDelaySeconds |   | The delay, between retries.  |
|  file |   | File, object specifying type and content, see File Object. This is required for task types that expect a script, having scriptable:true and an optionType of type:"file-content".  |

In [ ]:
##Initailizing Variables here
ACCESS_TOKEN="8d06730a-6478-403a-8c3e-837fd74c1b47"
#ACCESS_TOKEN="3f900cd0-d108-429a-88c3-592c713103fd"

API_URL="5xqm8quc36ij6dsubflp8u81r5.privatecloud.greenlake.hpe.com"

### Create now the 'Uptime Task'


example:
curl -XPOST "$serverUrl/api/tasks" \
  -H "Authorization: BEARER $accessToken" \
  -H "Content-Type: application/json" \
  -d '{"task": {
    "name": "cleanup tmp files",
    "taskType": {
      "code": "script"
    },
    "executeTarget": "resource",
    "file": {
      "sourceType": "local",
      "content": "rm -rf /var/www/app1/tmp/*\nrm -rf /var/www/app2/tmp/*"
    }
  }}'

In [ ]:
curl -XPOST "$API_URL/api/tasks" \
  -H "Authorization: BEARER $ACCESS_TOKEN" \
  -H "Content-Type: application/json" \
  -d '{"task": {
    "name": "Uptime Command",
    "taskType": {
      "code": "script"
    },
    "executeTarget": "resource",
    "file": {
      "sourceType": "local",
      "content": "uptime"
    }
  }}'

In [ ]:
curl -XPOST "$API_URL/api/tasks" \
  -H "Authorization: BEARER $ACCESS_TOKEN" \
  -H "Content-Type: application/json" \
  -d '{"task": {
    "name": "ls /tmp Command",
    "taskType": {
      "code": "script"
    },
    "executeTarget": "resource",
    "file": {
      "sourceType": "local",
      "content": "ls -al /tmp"
    }
  }}'

### Check Task is now listed.

In [ ]:
curl "$API_URL/api/tasks" \
-H "Authorization: BEARER $ACCESS_TOKEN" |jq -r

Check now the id of the uptime task. It should be 6. if not make sure to update the task execution cell with the correct one.
Please also check the instance id from the previous lab.

### Let's now execute the task on the instance we created earlier.


example:
curl -XPOST "$serverUrl/api/tasks/5/execute" \
  -H "Authorization: BEARER $accessToken" \
  -H "Content-Type: application/json" \
  -d '{"job":{
    "targetType": "instance",
    "instances": [1]
  }}'

In [ ]:
## Uptime Task execution Cell here

curl -XPOST "$API_URL/api/tasks/6/execute" \
  -H "Authorization: BEARER $ACCESS_TOKEN" \
  -H "Content-Type: application/json" \
  -d '{"job":{
    "targetType": "instance",
    "instances": [1]
  }}'

### Create now the 'Filecopy Task'

In [ ]:
##FileCopy Task creation Cell  here

curl -XPOST "$API_URL/api/tasks" \
  -H "Authorization: BEARER $ACCESS_TOKEN" \
  -H "Content-Type: application/json" \
  -d '{"task": {
    "name": "FileCreation",
    "taskType": {
      "code": "script"
    },
    "executeTarget": "resource",
    "file": {
      "sourceType": "local",
      "content": "echo > student {{STDID }}-$(date)/tmp/student {{STDID }}.md"
    }
  }}'

### Check Task is now listed.

In [ ]:
curl "$API_URL/api/tasks" \
-H "Authorization: BEARER $ACCESS_TOKEN" |jq -r

### Let's now execute the task on the instance we created earlier.

In [ ]:
## FileCreation Task execution Cell here

curl -XPOST "$API_URL/api/tasks/8/execute" \
  -H "Authorization: BEARER $ACCESS_TOKEN" \
  -H "Content-Type: application/json" \
  -d '{"job":{
    "targetType": "instance",
    "instances": [1]
  }}'

### We have created an ls -al /tmp task to actually provide you with  a way to chaeck the result of the previous task

Please run it to make sure your File Creation Task completed correctly.

In [ ]:
## ls -al /tmp Task execution Cell here

curl -XPOST "$API_URL/api/tasks/7/execute" \
  -H "Authorization: BEARER $ACCESS_TOKEN" \
  -H "Content-Type: application/json" \
  -d '{"job":{
    "targetType": "instance",
    "instances": [1]
  }}'

### Deleting Uptime and FileCopy Tasks now


example:
curl -XDELETE "$serverUrl/api/tasks/1" \
  -H "Authorization: BEARER $accessToken"

In [ ]:
## Tasks Deletion Cell here

curl -XDELETE "$API_URL/api/tasks/6" \
  -H "Authorization: BEARER $ACCESS_TOKEN" \

curl -XDELETE "$API_URL/api/tasks/6" \
  -H "Authorization: BEARER $ACCESS_TOKEN" \

### Check Tasks are no longer listed

In [ ]:
curl "$API_URL/api/tasks" \
-H "Authorization: BEARER $ACCESS_TOKEN" |jq -r

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#008000;"></i>&nbsp;&nbsp;Next Steps

Lab 3: Tasks

</br>
 <a href="2-WKSHP-WKSHP-VMAAS-Instance.ipynb" target="New" title="Back: Instance Creation"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#008000;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="4-WKSHP-WKSHP-VMAAS-Blueprint.ipynb#sc" target="New" title="Next:Blueprint Deployment"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#008000;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>


<h2>Next&nbsp;&nbsp;&nbsp;&nbsp;<a href="4-WKSHP-WKSHP-VMAAS-Blueprint.ipynb#sc" target="New" title="Next: Blueprint Deployment"><i class="fas fa-chevron-circle-right" style="color:#008000;"></i></a></h2>